دستور IN

این دو کد با هم برابر هستند

In [ ]:
SELECT 
	*
FROM
	Person.Person
WHERE
	BusinessEntityID = 5
	OR
	BusinessEntityID = 10
	OR
	BusinessEntityID = 20

In [ ]:
SELECT 
	*
FROM
	Person.Person
WHERE
	BusinessEntityID IN (5, 10, 20)

دستور NOT IN

این دو کد با هم برابر هستند

In [ ]:
SELECT 
	*
FROM
	Person.Person
WHERE
	BusinessEntityID <> 5
	AND
	BusinessEntityID <> 10
	AND
	BusinessEntityID <> 20

In [ ]:
SELECT 
	*
FROM
	Person.Person
WHERE
	BusinessEntityID NOT IN (5, 10, 20)

Sub Query

نوع اول

Self Contained 

Scalar Valued


مثال: مقدار فروش سالاه + درصد فروش هر سال از کل فروش

In [ ]:
SELECT 
	YEAR(SOH.OrderDate)					AS OredrYear,
	SUM(SOD.OrderQty * SOD.UnitPrice)	AS YearSales,
	(
		SELECT 
			SUM(OrderQty * UnitPrice)
		FROM
			Sales.SalesOrderDetail
	)									AS TotalSales,

	SUM(SOD.OrderQty * SOD.UnitPrice) / 
	(
		SELECT 
			SUM(OrderQty * UnitPrice)
		FROM
			Sales.SalesOrderDetail
	) * 100
FROM
	Sales.SalesOrderHeader AS SOH
INNER JOIN
	Sales.SalesOrderDetail AS SOD
ON
	SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY
	YEAR(SOH.OrderDate)
ORDER BY
	OredrYear

Sub Query

نوع دوم

Self Contained 

Multi Valued


مثال: لیست اشخاصی که کارمند هستند

این دو کویری با هم برابر هستند

In [ ]:
SELECT 
	P.*
FROM
	Person.Person AS P
INNER JOIN
	HumanResources.Employee AS E
ON
	P.BusinessEntityID = E.BusinessEntityID

In [ ]:
SELECT 
	*
FROM
	Person.Person AS P
WHERE
	BusinessEntityID IN (SELECT 
							BusinessEntityID
						FROM
							HumanResources.Employee)

مثال: لیست اشخاصی که کارمند نیستند

این دو کویری با هم برابر هستند

In [ ]:
SELECT 
	P.*
FROM
	Person.Person AS P
LEFT OUTER JOIN
	HumanResources.Employee AS E
ON
	P.BusinessEntityID = E.BusinessEntityID
WHERE
	E.BusinessEntityID IS NULL

In [ ]:
SELECT 
	*
FROM
	Person.Person AS P
WHERE
	BusinessEntityID NOT IN (SELECT 
								BusinessEntityID
							FROM
								HumanResources.Employee)

Sub Query

نوع سوم

Correlated

Scalar Valued


مثال: فروش سالانه، به تفکیک فصل

In [ ]:
SELECT 
	YEAR(SOH.OrderDate)					AS OrderYear,
	(
		SELECT 
			SUM(I_SOD.OrderQty * I_SOD.UnitPrice)
		FROM
			Sales.SalesOrderHeader AS I_SOH
		INNER JOIN
			Sales.SalesOrderDetail AS I_SOD
		ON
			I_SOH.SalesOrderID = I_SOD.SalesOrderID
		WHERE
				YEAR(I_SOH.OrderDate) = YEAR(SOH.OrderDate)
			AND DATEPART(QUARTER, I_SOH.OrderDate) = 1
	) AS Q1_Sales,
	(
		SELECT 
			SUM(I_SOD.OrderQty * I_SOD.UnitPrice)
		FROM
			Sales.SalesOrderHeader AS I_SOH
		INNER JOIN
			Sales.SalesOrderDetail AS I_SOD
		ON
			I_SOH.SalesOrderID = I_SOD.SalesOrderID
		WHERE
				YEAR(I_SOH.OrderDate) = YEAR(SOH.OrderDate)
			AND DATEPART(QUARTER, I_SOH.OrderDate) = 2
	) AS Q2_Sales,
	(
		SELECT 
			SUM(I_SOD.OrderQty * I_SOD.UnitPrice)
		FROM
			Sales.SalesOrderHeader AS I_SOH
		INNER JOIN
			Sales.SalesOrderDetail AS I_SOD
		ON
			I_SOH.SalesOrderID = I_SOD.SalesOrderID
		WHERE
				YEAR(I_SOH.OrderDate) = YEAR(SOH.OrderDate)
			AND DATEPART(QUARTER, I_SOH.OrderDate) = 3
	) AS Q3_Sales,
	(
		SELECT 
			SUM(I_SOD.OrderQty * I_SOD.UnitPrice)
		FROM
			Sales.SalesOrderHeader AS I_SOH
		INNER JOIN
			Sales.SalesOrderDetail AS I_SOD
		ON
			I_SOH.SalesOrderID = I_SOD.SalesOrderID
		WHERE
				YEAR(I_SOH.OrderDate) = YEAR(SOH.OrderDate)
			AND DATEPART(QUARTER, I_SOH.OrderDate) = 4
	) AS Q4_Sales
FROM
	Sales.SalesOrderHeader AS SOH
INNER JOIN
	Sales.SalesOrderDetail AS SOD
ON
	SOH.SalesOrderID = SOD.SalesOrderID
GROUP BY
	YEAR(SOH.OrderDate)
ORDER BY
	OrderYear

Sub Query

نوع چهارم

Correlated

Set Valued


مثال: لیست افرادی که کارمند هستند

+ EXISTS

In [ ]:
SELECT 
	*
FROM
	Person.Person AS P
WHERE
	EXISTS (SELECT  
				*
			FROM
				HumanResources.Employee AS E
			WHERE
				P.BusinessEntityID = E.BusinessEntityID)


مثال: لیست افرادی که کارمند نیستند

+ NOT EXISTS

In [ ]:
SELECT 
	*
FROM
	Person.Person AS P
WHERE
	NOT EXISTS (SELECT 
					*
				FROM
					HumanResources.Employee AS E
				WHERE
					P.BusinessEntityID = E.BusinessEntityID)

مثال: دو مشتری برتر یک سال مشخص

In [ ]:
SELECT TOP 2
	P.BusinessEntityID,
	P.FirstName,
	P.LastName,
	SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
FROM
	Sales.SalesOrderHeader AS SOH
INNER JOIN
	Sales.SalesOrderDetail AS SOD
ON	
	SOH.SalesOrderID = SOD.SalesOrderID
INNER JOIN
	Sales.Customer AS C
ON
	SOH.CustomerID = C.CustomerID
INNER JOIN
	Person.Person AS P
ON
	C.PersonID = P.BusinessEntityID
WHERE
	YEAR(SOH.OrderDate) = 2011
GROUP BY
	P.BusinessEntityID,
	P.FirstName,
	P.LastName
ORDER BY
	TotalSales DESC

مثال: دو مشتری برتر هر سال

+ CROSS APPLY

In [ ]:
SELECT 
	*
FROM
	Sales.vwSales_Years AS Y
CROSS APPLY
	(
		SELECT TOP 2
			P.BusinessEntityID,
			P.FirstName,
			P.LastName,
			SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
		FROM
			Sales.SalesOrderHeader AS SOH
		INNER JOIN
			Sales.SalesOrderDetail AS SOD
		ON	
			SOH.SalesOrderID = SOD.SalesOrderID
		INNER JOIN
			Sales.Customer AS C
		ON
			SOH.CustomerID = C.CustomerID
		INNER JOIN
			Person.Person AS P
		ON
			C.PersonID = P.BusinessEntityID
		WHERE
			YEAR(SOH.OrderDate) = Y.OrderYear
		GROUP BY
			P.BusinessEntityID,
			P.FirstName,
			P.LastName
		ORDER BY
			TotalSales DESC
	) AS TopCustomers
ORDER BY 
	Y.OrderYear

فانکشن:

دو مشتری برتر یک سال

پارامتر ورودی:  سال

In [ ]:
DROP FUNCTION IF EXISTS Sales.tvfTopCustomers
GO

CREATE FUNCTION Sales.tvfTopCustomers
(
    @Year INT,
	@N INT
)
RETURNS TABLE AS RETURN
(
    SELECT TOP(@N)
		P.BusinessEntityID,
		P.FirstName,
		P.LastName,
		SUM(SOD.OrderQty * SOD.UnitPrice) AS TotalSales
	FROM
		Sales.SalesOrderHeader AS SOH
	INNER JOIN
		Sales.SalesOrderDetail AS SOD
	ON	
		SOH.SalesOrderID = SOD.SalesOrderID
	INNER JOIN
		Sales.Customer AS C
	ON
		SOH.CustomerID = C.CustomerID
	INNER JOIN
		Person.Person AS P
	ON
		C.PersonID = P.BusinessEntityID
	WHERE
		YEAR(SOH.OrderDate) = @Year
	GROUP BY
		P.BusinessEntityID,
		P.FirstName,
		P.LastName
	ORDER BY
		TotalSales DESC
)

مثال: دو مشتری برتر هر سال

+ CROSS APPLY
+ ّFunction

In [ ]:
SELECT 
	*
FROM
	Sales.vwSales_Years AS Y
CROSS APPLY
	Sales.tvfTopCustomers(Y.OrderYear, 2) AS  TC
ORDER BY 
	Y.OrderYear

مثال: دو مشتری برتر هر سال

+  OUTER APPLY
+ ّFunction

In [ ]:
WITH Years
AS
(
			SELECT 2010 AS OrderYear
	UNION	SELECT 2011 AS OrderYear
	UNION	SELECT 2012 AS OrderYear
	UNION	SELECT 2013 AS OrderYear
	UNION	SELECT 2014 AS OrderYear
	UNION	SELECT 2015 AS OrderYear

)

SELECT 
	*
FROM
	Years AS Y
OUTER APPLY
	Sales.tvfTopCustomers(Y.OrderYear, 2) AS  TC
ORDER BY 
	Y.OrderYear